<a href="https://colab.research.google.com/github/ishaanamahajan/Self-Consistency-SemCoder-COMS6998/blob/main/semcoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/ARiSE-Lab/SemCoder.git

Cloning into 'SemCoder'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 167 (delta 87), reused 127 (delta 51), pack-reused 0 (from 0)
Receiving objects: 100% (167/167), 2.73 MiB | 6.14 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [3]:
%cd SemCoder

/content/SemCoder


In [4]:
import yaml

# Load the environment.yml file
with open("environment.yml", "r") as file:
    env = yaml.safe_load(file)

# Extract pip dependencies
pip_dependencies = []
for dep in env["dependencies"]:
    if isinstance(dep, dict) and "pip" in dep:
        pip_dependencies.extend(dep["pip"])

# Save to requirements.txt
with open("requirements.txt", "w") as req_file:
    req_file.write("\n".join(pip_dependencies))

print("requirements.txt created successfully!")


requirements.txt created successfully!


In [5]:

!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 93.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from transformers import pipeline
import torch

generator = pipeline(
  model="semcoder/semcoder_s_1030",
  task="text-generation",
  torch_dtype=torch.float16,
  device_map="auto",
)

# Generate Code

CODEGEN_REQUEST = """You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable <Code> according to <NL_Description>

<NL_Description>
{desc}

<Code>
"""
desc = """You are tasked with implementing a Python class that simulates a simple version of a "To-Do List" application. The class should have the following functionalities:
1. Add a new task to the to-do list.
2. Mark a task as completed.
3. Display all tasks in the to-do list.
4. Display only the incomplete tasks in the to-do list.
"""

prompt = CODEGEN_REQUEST.format(desc=desc)
result = generator(prompt, max_length=2048, num_return_sequences=1, temperature=0.0)
code = result[0]["generated_text"].split("```python")[1].split("```")[0]
print(code)

# Understand Code with Monologues

FWD_MNL_REQUEST = """Simulate the Execution: You are given a Python function and an assertion containing a function input. Complete the assertion containing the execution output corresponding to the given input in [ANSWER] and [/ANSWER] tags.
{code}
"""

tests = """
todo_list = ToDoList()
todo_list.add_task("Buy groceries")
todo_list.add_task("Complete assignment")
todo_list.mark_completed("Buy groceries")
assert todo_list.tasks == ???
"""
code += tests
prompt = FWD_MNL_REQUEST.format(code=code)
result = generator(prompt, max_length=2048, num_return_sequences=1, temperature=0.0)
print(result[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        self.tasks.append({'task': task, 'completed': False})

    def mark_completed(self, task):
        for t in self.tasks:
            if t['task'] == task:
                t['completed'] = True
                break

    def display_tasks(self):
        for t in self.tasks:
            status = 'Completed' if t['completed'] else 'Incomplete'
            print(f"Task: {t['task']}, Status: {status}")

    def display_incomplete_tasks(self):
        for t in self.tasks:
            if not t['completed']:
                print(f"Task: {t['task']}, Status: Incomplete")

Simulate the Execution: You are given a Python function and an assertion containing a function input. Complete the assertion containing the execution output corresponding to the given input in [ANSWER] and [/ANSWER] tags.

class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        

In [7]:
## Adding PL to NL conversion


from transformers import pipeline
import torch

# Load the model for code understanding
explainer = pipeline(
    model="semcoder/semcoder_s_1030",
    task="text2text-generation",
    torch_dtype=torch.float16,
    device_map="auto",
)

# Define the code to explain
code = """
class ToDoList:
    def __init__(self):
        self.tasks = {}

    def add_task(self, task):
        self.tasks[task] = False

    def mark_completed(self, task):
        if task in self.tasks:
            self.tasks[task] = True

    def display_tasks(self):
        for task, completed in self.tasks.items():
            print(f"{task}: {'Completed' if completed else 'Incomplete'}")

    def display_incomplete_tasks(self):
        for task, completed in self.tasks.items():
            if not completed:
                print(task)
"""

# Prompt for explaining the code
EXPLAIN_REQUEST = """You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable explanations for code.

<Code>
{code}

<NL_Explanation>
"""
prompt = EXPLAIN_REQUEST.format(code=code)

# Generate natural language explanation
result = explainer(prompt, max_length=2048, num_return_sequences=1, temperature=0.0)
explanation = result[0]["generated_text"]
print(explanation)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The model 'LlamaForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable explanations for code.

<Code>

class ToDoList:
    def __init__(self):
        self.tasks = {}

    def add_task(self, task):
        self.tasks[task] = False

    def mark_completed(self, task):
        if task in self.tasks:
            self.tasks[task] = True

    def display_tasks(self):
        for task, completed in self.tasks.items():
            print(f"{task}: {'Completed' if completed else 'Incomplete'}")

    def display_incomplete_tasks(self):
        for task, completed in self.tasks.items():
            if not completed:
                print(task)


<NL_Explanation>

The `ToDoList` class is a simple implementation of a to-do list. It allows users to add tasks, mark them as completed, and display the current tasks. Here's a breakdown of how it works:

- The `__init__` method initializes an empty dictionary `self.tasks` to store tasks. Each task is a key in the dictionar

In [1]:
# Testing NL to PL to NL, and back to PL

from transformers import pipeline
import torch

generator = pipeline(
  model="semcoder/semcoder_s_1030",
  task="text-generation",
  torch_dtype=torch.float16,
  device_map="auto",
)

# Generate Code

CODEGEN_REQUEST = """You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable <Code> according to <NL_Description>

<NL_Description>
{desc}

<Code>
"""
desc = """You are tasked with implementing a Python class that simulates a simple version of a "To-Do List" application. The class should have the following functionalities:
The `ToDoList` class is a simple implementation of a to-do list. It allows users to add tasks, mark them as completed, and display the current tasks. Here's a breakdown of how it works:

- The `__init__` method initializes an empty dictionary `self.tasks` to store tasks. Each task is a key in the dictionary, and its value is a boolean indicating whether the task is completed (`True`) or not (`False`).

- The `add_task` method takes a task as input and adds it to the `self.tasks` dictionary with a value of `False`, indicating it's not completed yet.

- The `mark_completed` method takes a task as input and sets its value in the `self.tasks` dictionary to `True`, indicating it's completed.

- The `display_tasks` method iterates over the `self.tasks` dictionary and prints each task along with its completion status.

- The `display_incomplete_tasks` method iterates over the `self.tasks` dictionary and prints only the tasks that are not completed (i.e., the ones with a value of `False`).
"""

prompt = CODEGEN_REQUEST.format(desc=desc)
result = generator(prompt, max_length=2048, num_return_sequences=1, temperature=0.0)
code = result[0]["generated_text"].split("```python")[1].split("```")[0]
print(code)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



class ToDoList:
    def __init__(self):
        self.tasks = {}

    def add_task(self, task):
        if task not in self.tasks:
            self.tasks[task] = False
        else:
            print(f"Task '{task}' already exists.")

    def mark_completed(self, task):
        if task in self.tasks:
            self.tasks[task] = True
        else:
            print(f"Task '{task}' does not exist.")

    def display_tasks(self):
        for task, completed in self.tasks.items():
            status = "Completed" if completed else "Incomplete"
            print(f"Task: {task}, Status: {status}")

    def display_incomplete_tasks(self):
        for task, completed in self.tasks.items():
            if not completed:
                print(f"Task: {task}, Status: Incomplete")

